In [2]:
# Hopkins CV group
# face detection 
# eye detection 
# iris detection 
# Writen by Jinxin
# 2018.10.11
import numpy as np
import matplotlib
import cv2
from PIL import Image
import glob
# PCA 

In [5]:
# collect face image data 
# process the PCA data 
def data_collection():
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    #eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    count = 0
    #glasses_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
    while(True):
        ret, frame = cap.read()
        # set the size of the frame
        ##########

        ##########
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray, 1.06, 10)


        # detect the face on the left side 

        for (x,y,w,h) in faces:
            cv2.rectangle(gray,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            count = count+1
        
            dir  = "train_data/Norm/Norm_%d.jpg" % count
            cv2.imwrite(dir,roi_gray)

        cv2.imshow('frame',gray)

    # when we press q on the keyborad, quit the program
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()

In [6]:
data_collection()

In [5]:
# resize the data 
# read all the image in file 
def data_resize():
    image_list = []
    for filename in glob.glob('train_data/CNN_train/*.jpg'): #assuming gif
        im=Image.open(filename)
        image_list.append(im)
    # resize all the image and saved in Jinxin_resized
    count = 0
    for image in image_list:

        count = count+1
        image = np.array(image);
        resized_image = cv2.resize(image, (100, 100))
        resized_image = resized_image[5:95,5:95]
        #print (resized_image.shape)
        dir  = "train_data/CNN_train_resize/resized_%d.jpg" % count
        cv2.imwrite(dir,resized_image)


In [6]:
# 0-1267 not Jinxin 1268-2308 Jinxin left 2309-2975 not Jinxin
data_resize()

In [1]:
# PCA
def zeroMean(dataMat):          
    meanVal=np.mean(dataMat,axis=0)       
    newData=dataMat-meanVal    
    return newData,meanVal 
def pca(dataMat,n):    
    newData,meanVal=zeroMean(dataMat) 
    # print (newData.shape)
    covMat=np.cov(newData,rowvar=0)        
    eigVals,eigVects=np.linalg.eig(np.mat(covMat)) 
    eigValIndice=np.argsort(eigVals)        
    n_eigValIndice=eigValIndice[-1:-(n+1):-1]   
    n_eigVect=eigVects[:,n_eigValIndice]        
    lowDDataMat=newData*n_eigVect              
    reconMat=(lowDDataMat*n_eigVect.T)+meanVal   
    return lowDDataMat, reconMat

In [2]:
# do PCA and get eigen_face
# read the train data
image_list = []
for filename in glob.glob('train_data/Jinxin_resized/*.jpg'): #assuming gif
    im=Image.open(filename)
    image_list.append(im)
    
N = len(image_list)
im = np.array(im)
[H,L] = im.shape
index = 0
dataMat = []

for image in image_list:
    
    image = np.array(image)
    dataMat.append(np.reshape(image,(H*L)))
    index = index+1

dataMat = np.array(dataMat).T
n = 50
eigen_face, reconMat = pca(dataMat,n)

print (eigen_face.shape)
print (reconMat.shape)


NameError: name 'glob' is not defined

In [14]:
labels = np.zeros(2975)
labels[0:1266] = 0
labels[1267:2307] = 1
labels[2308:2975] = 0
labels.shape

(2975,)

In [3]:
# Train CNN model
import Matrix_CV_ML3D as DImage
#import Matrix_CV_ML_realtime as RImage
from keras import backend as K
import numpy as np
from keras.models import load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import cifar10
import cv2
import numpy as np
import math

K.set_image_dim_ordering('th')
x = DImage.Matrix_CV_ML3D("Train_data/CNN_train_resize",100,100)
x.build_ML_matrix()
# 0-1267 not Jinxin 1268-2308 Jinxin left 2309-2975 not Jinxin

labels = np.zeros(2975)
labels[0:1266] = 0
labels[1267:2307] = 1
labels[2308:2974] = 0
x.labels = labels
y = np_utils.to_categorical(x.labels)
x = x.global_matrix
x = x.astype('float32')/255

model = Sequential()

model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(3,100,100)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
   
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
 
# 8. Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])





# 9. Fit model on training data
model.fit(x, y, batch_size=32, nb_epoch=80,  shuffle = True, verbose=2)
 
# 10. Evaluate model on test data
score = model.evaluate(x, y, verbose=0)

# Save Training Model

model.save('model.h5')  # creates a HDF5 file 'my_model.h5'

C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/80
 - 253s - loss: 1.1552 - acc: 0.6457
Epoch 2/80
 - 270s - loss: 0.6442 - acc: 0.6484
Epoch 3/80
 - 287s - loss: 0.6296 - acc: 0.6555
Epoch 4/80
 - 291s - loss: 0.6132 - acc: 0.6733
Epoch 5/80
 - 289s - loss: 0.6009 - acc: 0.6817
Epoch 6/80
 - 290s - loss: 0.5917 - acc: 0.6820
Epoch 7/80
 - 291s - loss: 0.5779 - acc: 0.6877
Epoch 8/80
 - 289s - loss: 0.5637 - acc: 0.6968
Epoch 9/80
 - 293s - loss: 0.5496 - acc: 0.7156
Epoch 10/80
 - 291s - loss: 0.5323 - acc: 0.7247
Epoch 11/80
 - 290s - loss: 0.5060 - acc: 0.7398
Epoch 12/80
 - 290s - loss: 0.4851 - acc: 0.7580
Epoch 13/80
 - 290s - loss: 0.4556 - acc: 0.7795
Epoch 14/80
 - 284s - loss: 0.4335 - acc: 0.7983
Epoch 15/80
 - 289s - loss: 0.3895 - acc: 0.8222
Epoch 16/80
 - 289s - loss: 0.3701 - acc: 0.8272
Epoch 17/80
 - 283s - loss: 0.3358 - acc: 0.8501
Epoch 18/80
 - 283s - loss: 0.3133 - acc: 0.8639
Epoch 19/80
 - 281s - loss: 0.2898 - acc: 0.8726
Epoch 20/80
 - 282s - loss: 0.2923 - acc: 0.8706
Epoch 21/80
 - 283s - loss: 0

In [ ]:
# Train CNN model
import Matrix_CV_ML3D as DImage
#import Matrix_CV_ML_realtime as RImage
from keras import backend as K
import numpy as np
from keras.models import load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import cifar10
import cv2
import numpy as np
import math

In [22]:
test = DImage.Matrix_CV_ML3D("test_data/CNN_test",100,100)
test.build_ML_matrix()
labels = np.zeros(399)
labels[0:399] = 1
test.labels = labels
y = np_utils.to_categorical(test.labels)
test = test.global_matrix
test = test.astype('float32')/255

(1, 3, 100, 100)(1, 3, 100, 100)
(1, 3, 100, 100)(2, 3, 100, 100)
(1, 3, 100, 100)(3, 3, 100, 100)
(1, 3, 100, 100)(4, 3, 100, 100)
(1, 3, 100, 100)(5, 3, 100, 100)
(1, 3, 100, 100)(6, 3, 100, 100)
(1, 3, 100, 100)(7, 3, 100, 100)
(1, 3, 100, 100)(8, 3, 100, 100)
(1, 3, 100, 100)(9, 3, 100, 100)
(1, 3, 100, 100)(10, 3, 100, 100)
(1, 3, 100, 100)(11, 3, 100, 100)
(1, 3, 100, 100)(12, 3, 100, 100)
(1, 3, 100, 100)(13, 3, 100, 100)
(1, 3, 100, 100)(14, 3, 100, 100)
(1, 3, 100, 100)(15, 3, 100, 100)
(1, 3, 100, 100)(16, 3, 100, 100)
(1, 3, 100, 100)(17, 3, 100, 100)
(1, 3, 100, 100)(18, 3, 100, 100)
(1, 3, 100, 100)(19, 3, 100, 100)
(1, 3, 100, 100)(20, 3, 100, 100)
(1, 3, 100, 100)(21, 3, 100, 100)
(1, 3, 100, 100)(22, 3, 100, 100)
(1, 3, 100, 100)(23, 3, 100, 100)
(1, 3, 100, 100)(24, 3, 100, 100)
(1, 3, 100, 100)(25, 3, 100, 100)
(1, 3, 100, 100)(26, 3, 100, 100)
(1, 3, 100, 100)(27, 3, 100, 100)
(1, 3, 100, 100)(28, 3, 100, 100)
(1, 3, 100, 100)(29, 3, 100, 100)
(1, 3, 100, 100)(30, 3,

(1, 3, 100, 100)(245, 3, 100, 100)
(1, 3, 100, 100)(246, 3, 100, 100)
(1, 3, 100, 100)(247, 3, 100, 100)
(1, 3, 100, 100)(248, 3, 100, 100)
(1, 3, 100, 100)(249, 3, 100, 100)
(1, 3, 100, 100)(250, 3, 100, 100)
(1, 3, 100, 100)(251, 3, 100, 100)
(1, 3, 100, 100)(252, 3, 100, 100)
(1, 3, 100, 100)(253, 3, 100, 100)
(1, 3, 100, 100)(254, 3, 100, 100)
(1, 3, 100, 100)(255, 3, 100, 100)
(1, 3, 100, 100)(256, 3, 100, 100)
(1, 3, 100, 100)(257, 3, 100, 100)
(1, 3, 100, 100)(258, 3, 100, 100)
(1, 3, 100, 100)(259, 3, 100, 100)
(1, 3, 100, 100)(260, 3, 100, 100)
(1, 3, 100, 100)(261, 3, 100, 100)
(1, 3, 100, 100)(262, 3, 100, 100)
(1, 3, 100, 100)(263, 3, 100, 100)
(1, 3, 100, 100)(264, 3, 100, 100)
(1, 3, 100, 100)(265, 3, 100, 100)
(1, 3, 100, 100)(266, 3, 100, 100)
(1, 3, 100, 100)(267, 3, 100, 100)
(1, 3, 100, 100)(268, 3, 100, 100)
(1, 3, 100, 100)(269, 3, 100, 100)
(1, 3, 100, 100)(270, 3, 100, 100)
(1, 3, 100, 100)(271, 3, 100, 100)
(1, 3, 100, 100)(272, 3, 100, 100)
(1, 3, 100, 100)(273

In [23]:
model = load_model('model.h5')
pred  = model.predict(test)
predicted = np.argmax(pred,axis=1)


In [24]:
predicted


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,

In [5]:
# PCA face detection in real time half and half 
up_threshold_Jinxin = 13002236.8621
down_threshold_Jinxin = 8866691.00754
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
#glasses_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
while(True):
    ret, frame = cap.read()
    # set the size of the frame
    ##########
    
    ##########
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    [H,L] = gray.shape
    gray1 = gray[0:H+1,0:np.int16(round(L/2.))]
    gray2 = gray[0:H+1,np.int16(round(L/2.)):L+1]
    
    faces1 = face_cascade.detectMultiScale(gray1, 1.1, 10)
    faces2 = face_cascade.detectMultiScale(gray2, 1.1, 10)
    
    # detect the face on the left side 
    for (x,y,w,h) in faces1:
        cv2.rectangle(gray1,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        resized_image = cv2.resize(roi_gray, (100, 100))
        resized_image = resized_image[5:95,5:95]
        resized_image_v = np.reshape(resized_image,(90*90,1))
        # do PCA
        Wt = (eigen_face.T)*resized_image_v
        Wt_norm=np.linalg.norm(Wt)
        if Wt_norm< up_threshold_Jinxin and Wt_norm>down_threshold_Jinxin:
            cv2.putText(gray1,'This is Jinxin!',(50,20),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,255),2)
            
        else:
            cv2.putText(gray1,"This is not Jinxin!", (50,20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2,(0,0,255),2)
        print (Wt_norm) 
        
        
        # we can also use eye detetion to creat eye ball control
        #eyes = eye_cascade.detectMultiScale(roi_gray)
        #glass = glasses_cascade.detectMultiScale(roi_gray)
        #for (ex,ey,ew,eh) in eyes:
        #    cv2.rectangle(roi_gray,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        #for (gx,gy,gw,gh) in glass:
        #    cv2.rectangle(roi_gray,(gx,gy),(gx+gw,gy+gh),(0,255,0),2)
        
        #
    cv2.imshow('frame1',gray1)
    # detect the face on the right side 
    for (x,y,w,h) in faces2:
        cv2.rectangle(gray2,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]    
        # do PCA
        resized_image = cv2.resize(roi_gray, (100, 100))
        resized_image = resized_image[5:95,5:95]
        resized_image_v = np.reshape(resized_image,(90*90,1))
        # get projection using eigen_face
        Wt = (eigen_face.T)*resized_image_v
        Wt_norm=np.linalg.norm(Wt)
        if Wt_norm< up_threshold_Jinxin and Wt_norm>down_threshold_Jinxin:
            cv2.putText(gray2,'This is Jinxin!',(50,20),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,255),2)
        else:
            cv2.putText(gray2,"This is not Jinxin!", (50,20), cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,255),2)
        print (Wt_norm) 
        
        
        #eyes = eye_cascade.detectMultiScale(roi_gray)
        #glass = glasses_cascade.detectMultiScale(roi_gray)
        #for (ex,ey,ew,eh) in eyes:
        #    cv2.rectangle(roi_gray,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        #for (gx,gy,gw,gh) in glass:
        #    cv2.rectangle(roi_gray,(gx,gy),(gx+gw,gy+gh),(0,255,0),2)  
    
    cv2.imshow('frame2',gray2)
    
# when we press q on the keyborad, quit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
cap.release()
cv2.destroyAllWindows()

37224290.30818738
38539310.22206096
31886029.685174525
26096256.845490526
22001909.549223315
26344776.56892268
37173629.28321363
39801855.54711573
41047468.3313467
39666050.60845609
42259861.580352426
43482778.271820284
40821840.92028506
41355694.523906454
42330637.33456641
43059589.48646855
42702237.92725674
42790082.323968664
41829166.74658142
41412368.15063587
41607132.737018414
41061003.38331641
41045616.99140771
41081354.017587274
42071005.59914865
41572200.838539004
41178950.97631943
41847320.04762014
41130371.13619539
41076509.05534888
42104436.795254506
41510307.79857703
42527049.14471168
42510821.02783851
40633108.39011845
42185675.354092926
39947814.557294145
39515038.634884454
42411218.11102615
44211069.05959454
48937150.47511365
48011127.92131573
44761464.35729285
41844730.15490999
43609303.21589723
44434608.04324682
47662385.44537974
46341535.256090686
43474457.9303495
39871623.88455044
34392101.11215353
33308614.74850567
41172284.63731089
46798069.74257279
47451692.646421

In [8]:
# PCA face detection in real time realtime location of Jinxin
up_threshold_Jinxin = 59051554.81861222
down_threshold_Jinxin = 45343522.16734847
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
#glasses_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
while(True):
    ret, frame = cap.read()
    # set the size of the frame
    ##########
    
    ##########
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    [H,L] = gray.shape
    

    faces = face_cascade.detectMultiScale(gray, 1.1, 10)

    
    # detect the face on the left side 
    for (x,y,w,h) in faces:
        cv2.rectangle(gray,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        resized_image = cv2.resize(roi_gray, (100, 100))
        resized_image = resized_image[5:95,5:95]
        resized_image_v = np.reshape(resized_image,(90*90,1))
        # do PCA
        Wt = (eigen_face.T)*resized_image_v
        Wt_norm=np.linalg.norm(Wt)
        print (Wt_norm)
        if Wt_norm< up_threshold_Jinxin and Wt_norm>down_threshold_Jinxin:
            # publish the topic 
            location = (x+w)/2.
            # pubilsh the topic
            distance = w/100.
            cv2.putText(gray,'Found Jinxin!',(50,20),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,255),2)
            dir_location = 'loaction:%d distance %d'% (location, distance)
            cv2.putText(gray,dir_location,(50,150),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,255),2)
            
        else:
            cv2.putText(gray,"Jinxin not here!", (50,20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2,(0,0,255),2)
            
        
        
        # we can also use eye detetion to creat eye ball control
        #eyes = eye_cascade.detectMultiScale(roi_gray)
        #glass = glasses_cascade.detectMultiScale(roi_gray)
        #for (ex,ey,ew,eh) in eyes:
            #cv2.rectangle(roi_gray,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        #for (gx,gy,gw,gh) in glass:
        #    cv2.rectangle(roi_gray,(gx,gy),(gx+gw,gy+gh),(0,255,0),2)
        
        #
    cv2.imshow('frame1',gray)
    
    
# when we press q on the keyborad, quit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
cap.release()
cv2.destroyAllWindows()

54241170.68913066
54988277.9483564
57540747.65291509
59583689.123101905
60817989.64726729
58969331.49396598
59898942.55788221
56449122.806400254
52729577.709427014
50223372.31671866
53155431.48050279
55862353.07883619
49978459.90719871
58853570.47756046
52075230.03382456
65928487.63460465
51640127.71949667
64220666.80967053
52390609.43862956
55987734.457058325
51501578.48655526
55946638.122037925
51586861.262024224
55865260.41972401
53913826.1428596
56547476.32554979
55916132.01951111
55456464.993562624
56076047.87114789
58034725.55744267
59181751.19661151
59701876.92494054
59491035.953927904
61080960.856742814
60542935.25634799
60955992.77619128
60900802.33738325
61034183.78841595
61654391.510721795
61142115.37054898
58378160.04309782
61405565.53139923
61107582.05065348
60256707.262971774
60503526.45447737
61071143.903952956
60621021.53018825
60091406.735680565
59301752.65523489
60070050.56877127
59858943.50003792
60383955.10987983
60336159.54432315
60828352.64883577
59845027.56280430

58817050.22644768
25541008.49611568
56280958.11319092
25183633.28153718
57459692.43329874
23355799.914915413
56209889.98159438
25859356.879041284
55375887.91165769
53282814.24235667
24601821.2263469
54544509.715405144
57308631.411282666
24903443.649055254
56410578.84913569
23697040.387777057
57660260.80299467
26090072.648082595
57950741.05688244
24595909.148665115
54010037.38635622
24649481.357658684
58415049.73428604
25218595.833593585
59247595.1610444
24732598.340752594
57047221.06637968
25461918.51614418
57843685.685466714
26907735.53244516
59839573.653320156
26952759.79630948
60594370.21261
24409328.66055133
60992787.977250464
24853248.267212518
60049136.58139885
24453488.864875168
60031419.08469072
25054046.63682151
56532203.77250086
24503338.8999168
57451741.20639361
23823359.72643645
57797951.64459093
26418617.974719696
56221126.68031469
27221809.847281687
57526850.771263406
25643054.683284696
54990480.8430859
26233850.052858442
53424505.36925538
54592272.21067887
53424505.36925

51900896.916282825
28498125.466673862
51840084.47176911
51540124.1099345
50413368.29374879
28172575.452900033
52375719.62017116
52665576.248361796
54068440.82460694
27549440.228375353
54604428.24773578
28352045.04340671
53292664.92583731
53113140.76405391
25759933.235044923
53049708.96862506
25727500.736569602
52705107.819236085
28485629.711075656
52489867.40439594
27119852.990054995
51042993.44067574
28111545.808695804
51670820.06451004
27761417.039678838
52655705.983091265
26746961.20380316
51606876.28843719
28406901.139862828
51315402.59465748
26695754.20415469
51339930.045930326
28109229.443486545
51353289.57648529
27931179.748775456
51169249.21753494
28771734.285995442
51295603.87038987
51075797.06820521
51033835.66522516
27733286.56539405
50813727.5810735
28558782.89742797
50551488.13839465
50203144.727628894
27459726.781265408
51102998.01942832
28525736.302643128
50373108.181897946
27988348.466237623
51165893.494840086
28613831.06902798
52268815.99176978
50163609.82602963
529428

54568197.32147865
53130979.70977696
55128087.51649497
56101492.991428055
55893760.96134429
55398486.72432452
54401810.44212745
53313790.836870894
55224274.88467584
53973508.90710445
54846033.0625225
54176053.598224394
54073386.62194492
55001048.685243994
53351023.77730464
19670136.531656805
53358874.203490704
53389673.57872008
55135260.87863455
55715642.23191544
54387946.15997945
53573529.86261669
55015601.123634376
54120547.7467269
53062933.30651743
54060306.40925684
53194934.6684347
53796593.61748445
53311115.312087044
53510222.95981636
55029962.268181935
54778587.64269676
52545812.79651454
53394535.56912996
54232495.88175836
54053632.21580713
54927297.739825904
53379961.55203553
52831259.985273175
52076601.09919011
53289442.377307594
52437197.585355826
52705185.93404024
52709803.80315706
53846283.449128345
51960076.83657966
50912081.38988876
50752789.44340493
17690761.184398636
49252094.194031
48236027.544540815
48128757.87423138
47570546.82446439
46887198.62975739
49066996.23214169

37741135.90715011
41304791.48106771
40968253.90696482
40584168.53287378
39920539.634593055
38574428.385920025
41460271.53523159
38692912.33672074
39250058.44758572
34821016.808544986
40201678.234216996
40978492.003332816
40399102.73944962
41334052.11046461
40175224.76029814
38007198.25332093
39336452.0406006
41238726.43915318
40215309.89206261
41784945.22431268
42489938.21247266
40087109.09379119
37311164.9376643
32849896.23211334
40724749.06609018
42158329.482965484
40919920.69971156
42929716.67934912
40942174.740455925
40977619.44568689
43249027.31824556
41615854.563843094
40754081.04479524
41867942.135901645
40540610.85672437
37343627.916558914
39554133.25867043
35981892.5480814
34094830.034537554
40108283.65940121
34499109.33661702
41046395.79100828
41298832.736647904
41841652.317884296
41400149.938770905
41378331.71700115
40605610.78785042
41120945.6138229
38772678.27636991
40050883.67133612
39370477.44782198
40636680.910246305
37580485.89604494
38280847.42121659
35917176.46547599

35269759.51406591
36746911.83230427
35950013.57576081
34854861.294633634
35066953.38884262
34706274.793269284
33257304.207198128
34337059.08039387
31891117.79208989
32109856.68905731
33422250.35909323
34704734.84280964
33912044.67767235
34900241.38656875
34424798.7939358
35900957.29900106
34569323.96000412
34943730.31752609
36741777.58203751
36245331.824402995
34812867.01749432
37353270.85790275
36524783.94566583
36112464.96817021
34232281.247845404
35312104.97888322
36796573.53719913
37416295.81170566
37939957.69257416
36314132.73894425
36147970.81396874
35228391.144974895
34804520.75294643
37009840.767886475
38266051.95337645
38102968.293907635
37391405.83879856
37987142.34433317
34974906.951286726
34541427.60090237
34283444.715949744
34024888.308206275
32275349.582997464
37118385.26737318
37440929.47920785
35434926.204473
35822133.995723195
37816556.1483934
39469597.191627294
38865828.799719565
37787776.37611101
34423179.845361136
35665702.85008393
31541668.143669475
34719094.276404

33251982.73833974
32718540.27677938
32912965.221645467
32220771.169546083
33346738.02991613
35721430.601913124
35136251.35143448
35663679.63000929
37080292.84786772
35930605.479422346
33641501.91865984
32778676.68486907
34829625.23800598
33947322.149016246
36112163.30482437
34724679.54836989
35057276.3785005
34941607.09311474
36263782.22581334
37396947.02717938
37749276.279985525
37300142.252973005
34849490.902986385
35445309.349810965
35106983.65725721
36539067.40075484
36543124.66717272
34225545.3582667
34544004.0297822
35936466.53281952
34319712.63347827
37831159.854764126
38032462.53040756
34534609.04982238
34582957.60433792
33708025.24504358
33772048.56801976
34249116.190458335
33573065.09097177
33364903.88511904
34586096.97691144
33945749.51546306
34802665.90461267
33728209.20390162
30793756.598318603
35633775.63193946
35654536.62896657
34435589.086870916
34961076.954616435
35396158.66305199
33615791.89621332
32380645.126414444
36545829.812257424
35627560.37965736
36663576.126599

38820886.46535812
36208502.570733316
36168676.775468335
36129322.83386275
35960224.23683968
36460369.73657239
34443527.06472371
35268345.30885924
33857855.35216006
34877188.82121848
38307895.09968221
37164218.446832865
35108010.69971054
32778550.692036595
36398897.31309374
38884396.94239536
36788973.67666818
38524129.74973717
41382589.28029739
39739720.51729494
37638494.21118301
36301550.23248883
39594239.870124914
39625574.50268967
38908829.62340492
39634765.775287986
37988465.93557405
38721087.64591223
36600789.59207008
41257030.94003722
39467431.603821166
38306568.94979285
41581665.07545699
41428724.276855305
40011438.19246362
39887790.850389384
41044692.13944936
41017248.50878107
39867150.26281521
40930925.1980494
41067468.612963215
41681424.43679207
42518246.46966261
41628527.50742054
43426408.71530924
42649827.84324109
43013175.28427899
45431869.87031971
44356880.68548827
43496757.52466052
41380369.47926758
44521977.2649188
45493542.45637612
45337428.359233215
45056133.842649646


In [1]:
# eyeball and iris tracking 
import numpy as np
import cv2
import numpy as np
import matplotlib
import cv2
from PIL import Image
import glob

def sequential_label(binary_in):
    a = binary_in.shape
    a = np.array([int(v) for v in a])
    I = binary_in
    label = 0
    labeled_image = np.zeros((a[0],a[1]))
    record = []
    for i in range(1,a[0]):
        for j in range(1,a[1]):
            if I[i][j]!=0: 
                if labeled_image[i-1][j-1]!=0:
                    labeled_image[i][j] = labeled_image[i-1][j-1]

                elif labeled_image[i][j-1]==0 and labeled_image[i-1][j]==0:
                    label = label+1
                    labeled_image[i][j] = label
                elif labeled_image[i-1][j]!=0 and labeled_image[i][j-1]==0:
                    labeled_image[i][j] = labeled_image[i-1][j]

                elif labeled_image[i-1][j]==0 and labeled_image[i][j-1]!=0:
                    labeled_image[i][j] = labeled_image[i][j-1]    

                elif labeled_image[i-1][j]!=0 and labeled_image[i][j-1]!=0:

                    if labeled_image[i][j-1]==labeled_image[i-1][j]:
                        labeled_image[i][j] = labeled_image[i][j-1]

                    else: 
                        minValue = min(labeled_image[i][j-1],labeled_image[i-1][j])
                        maxValue = max(labeled_image[i][j-1],labeled_image[i-1][j])
                        labeled_image[i][j] = labeled_image[i][j-1]
                        # record equivalence of labels
                        labeled_image = np.where(labeled_image == minValue, maxValue, labeled_image)

    labelled_img = labeled_image
    # normalized
    
    max_labelled_img = labelled_img.max()
    labelled_img = (labelled_img*(255/max_labelled_img)).astype('uint8')
    
        
    return labelled_img

def moment_opencv(labelled_in):
    res = {}
    for label_idx in np.unique(labelled_in):
        if label_idx == 0:
            continue
        masked_in = (labelled_in==label_idx).astype(labelled_in.dtype)
        moments = cv2.moments(masked_in)
        res[label_idx] = [moments['m00'],moments['m01'],moments['m10'],
                          moments['m02'],moments['m11'],moments['m20'],
                          moments['mu02'],moments['mu11'],moments['mu20']]
    return res

def binarize(gray_in, threshold=128):
    #ret,thresh1 = cv2.threshold(img,threshold,255,cv.THRESH_BINARY)
    binary_image = np.where(gray_in>threshold,0,255).astype('uint8')
    return binary_image

def find_iris(eye_image):
    eye = np.array(eye_image)
    gray_eye = np.array(eye)
    [H,L] = eye_image.shape    
    scaler_x= H/40.
    scaler_y=L/40.
    gray_eye = cv2.resize(gray_eye,(40,40))
    #cv2.imshow('frame2',gray_eye)

    print (scaler_x,scaler_y)
    gray_eye = cv2.medianBlur(gray_eye, 5)
    gray_eye = binarize(gray_eye,50)
    label = sequential_label(gray_eye)
    cv2.imshow('frame3',label)
    if label.max() == 0:
        x_hat = 0
        y_hat = 0
        
    else:
        
        res = moment_opencv(label)
        list_A = []
        list_i = []
        count = 0
        for i in res:
            list_A.append(res[i][0])
            count = count+1
            list_i.append(i)
        index = np.where(list_A == max(list_A))
                
        index = np.array(index)       
        
    
        if index.shape == (1,0):
            index = 0
            
        ii = list_i[index]

        x_hat = round((float(res[ii][1])/float(res[ii][0]))*scaler_x)
        y_hat = round((float(res[ii][2])/float(res[ii][0]))*scaler_y)
        r = 5
        return x_hat,y_hat

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
cap = cv2.VideoCapture(0)
count = 0
while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray,1.1,10)
        print (eyes)
        for (ex,ey,ew,eh) in eyes:
#            ex_smallest = np.minimum(ex)
#           index_smallest = where(ex==ex_smallest)
            count_eye = count_eye +1
            if count_eye == 2:
                break
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            eye_img = roi_gray[ey:ey+eh,ex:ex+ew]
            #eye_color = roi_color[ey:ey+eh,ex:ex+ew]
            count= count +1
            eye_image = eye_img
            if eye_image is not None:
                try:
                    x_hat,y_hat = find_iris(eye_image)

                except:
                    print ('cant find iris!')
                else:                
                    r = 2
                    cv2.circle(roi_color, (np.int64(ex+x_hat), np.int64(ey+y_hat)), r, (0, 255, 0), 2)

                cv2.imshow('frame2',eye_image)
            
            
    cv2.imshow('frame1',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
[[23 39 53 53]]
1.325 1.325
cant find iris!
()
()
()


C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in double_scalars
C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


[[18 40 57 57]]
1.425 1.425
cant find iris!
[[36 42 55 55]]
1.375 1.375
cant find iris!
()
()
()
()
()
()
[[76 49 28 28]]
0.7 0.7
cant find iris!
[[78 47 29 29]]
0.725 0.725
cant find iris!
[[77 47 34 34]]
0.85 0.85
cant find iris!
[[78 49 31 31]]
0.775 0.775
cant find iris!
[[77 50 33 33]]
0.825 0.825
cant find iris!
[[83 52 29 29]]
0.725 0.725
cant find iris!
[[96 61 27 27]
 [37 46 33 33]]
0.675 0.675
cant find iris!
[[97 59 26 26]
 [40 43 37 37]]
0.65 0.65
cant find iris!
[[89 55 32 32]
 [35 41 32 32]]
0.8 0.8
cant find iris!
[[94 60 33 33]]
0.825 0.825
cant find iris!
[[90 57 29 29]
 [36 46 25 25]
 [32 36 44 44]]
0.725 0.725
cant find iris!
[[92 58 30 30]]
0.75 0.75
cant find iris!
[[91 58 32 32]
 [36 43 34 34]]
0.8 0.8
cant find iris!
[[94 60 28 28]]
0.7 0.7
cant find iris!
[[87 55 33 33]]
0.825 0.825
cant find iris!
[[95 62 25 25]]
0.625 0.625
cant find iris!
[[88 55 32 32]
 [32 41 34 34]]
0.8 0.8
cant find iris!
[[78 51 34 34]]
0.85 0.85
cant find iris!
[[91 57 25 25]]
0.625 0.6

In [4]:
a = np.array([1,2,3,2])
aa = np.sort(a)
aa
eyes = [[22, 33, 44, 55],
       [44, 55, 66, 77]]
eyes

[[22, 33, 44, 55], [44, 55, 66, 77]]

In [1]:

# import the necessary packages
import numpy as np

import cv2


cap = cv2.VideoCapture(0)
# load the image, clone it for output, and then convert it to grayscale


while(True):
    ret, frame = cap.read()
    # set the size of the frame
    cimg = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    # detect circles in the image
    circles = cv2.HoughCircles(cimg,cv2.HOUGH_GRADIENT,1,20,
                                param1=50,param2=30,minRadius=0,maxRadius=0)
    
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
        # draw the outer circle
        cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
        # draw the center of the circle
        cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
    
    cv2.imshow('detected circles',cimg)
    cv2.destroyAllWindows()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



KeyboardInterrupt: 

In [1]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
count = 0
while(True):
    ret, frame = cap.read()
    output = frame
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)

    # ensure at least some circles were found
    if circles is not None:
        # convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")

        # loop over the (x, y) coordinates and radius of the circles
        for (x, y, r) in circles:
            # draw the circle in the output image, then draw a rectangle
            # corresponding to the center of the circle
            cv2.circle(output, (x, y), r, (0, 255, 0), 4)
            cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)

    cv2.imshow('frame', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [25]:
# test find_iris use single image 
image_list = []
for filename in glob.glob('train_data/Jinxin_eye/*.jpg'): 
    im=Image.open(filename)
    image_list.append(im)
# resize all the image and saved in Jinxin_resized
for eye in image_list:
    eye = np.array(eye)
    gray_eye = np.array(eye)
    gray_eye = cv2.resize(gray_eye,(40,40))
    #cv2.imshow('frame2',gray_eye)
    gray_eye = cv2.medianBlur(gray_eye, 5)
    gray_eye = binarize(gray_eye,50)
    label = sequential_label(gray_eye)
    
    if label.max() == 0:
        print ('find nothing')
    else:
        res = moment_opencv(label)
        list_A = []
        list_i = []
        count = 0
        for i in res:
            list_A.append(res[i][0])
            count = count+1
            list_i.append(i)
        index = np.where(list_A == min(list_A))
                
        index = np.array(index)       
        
    
        if index.shape == (1,0):
            index = 0
            
        ii = list_i[index]

        x_hat = round(float(res[ii][1])/float(res[ii][0]))
        y_hat = round(float(res[ii][2])/float(res[ii][0]))
        r = 5
        cv2.circle(eye, (x_hat, y_hat), r, (0, 255, 0), 2)
    cv2.imshow('frame',eye)
    cv2.waitKey(0)
cv2.destroyAllWindows()
##################################### backup code Just in case 

NameError: name 'binarize' is not defined

In [43]:
# Image Segmentation with Watershed Algorithm
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

image_list = []
for filename in glob.glob('train_data/Jinxin_eye/*.jpg'): 
    im=Image.open(filename)
    image_list.append(im)
    
img = image_list[30];
img = np.array(img)
gray = img

img = cv2.imread('Screenshot1.png')
img = np.array(img)
gray = cv2.medianBlur(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),5)

ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg,sure_fg)
# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,0]

cv2.imshow('frame',img)

cv2.waitKey(0)
cv2.destroyAllWindows()



In [22]:
# face and eye detection example code 

import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

img = cv2.imread('heat.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.1, 10)
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# brigt object detection
'''
Author Jinxin Dong 
2018.8.6
'''
import cv2 
import numpy as np 
  
camera = cv2.VideoCapture(0) 
if (camera.isOpened()): 
    print('Open') 
else: 
    print('The camera havent been open') 
    
size = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)), 
    int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))) 
print('size:'+repr(size)) 
  
es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 4)) 
kernel = np.ones((5, 5), np.uint8) 
background = None
# test end 
while True: 

    grabbed, frame_lwpCV = camera.read() 
    gray_lwpCV = cv2.cvtColor(frame_lwpCV, cv2.COLOR_BGR2GRAY) 
    gray_lwpCV = cv2.GaussianBlur(gray_lwpCV, (21, 21), 0) 


    if  background is None: 
        background = gray_lwpCV 
        continue

    diff = cv2.absdiff(background, gray_lwpCV) 
    diff = cv2.threshold(diff, 150, 255, cv2.THRESH_BINARY)[1] 
    diff = cv2.dilate(diff, es, iterations=2) 

    image, contours, hierarchy = cv2.findContours(diff.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    for c in contours: 
        if cv2.contourArea(c) < 1500: 
            continue
        (x, y, w, h) = cv2.boundingRect(c) 
        cv2.rectangle(frame_lwpCV, (x, y), (x+w, y+h), (0, 255, 0), 2) 

        cv2.imshow('contours', frame_lwpCV) 
        cv2.imshow('dis', diff) 

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): 
        break
# When everything done, release the capture 
camera.release()  
cv2.destroyAllWindows()

Open
size:(640, 480)


In [17]:
# using ios web camera (take ipad as a camera) 

import cv2


cam_url='http://10.0.0.67:8081/video'
print cam_url
vc=cv2.VideoCapture(cam_url)
if vc.isOpened(): 
    rval, frame = vc.read()
else:
    rval = False

rval = True
while rval:
#    frame=cv2.resize(frame,(100,100)) 
    cv2.imshow("webcam test", frame)

    rval, frame = vc.read()
    
    key = cv2.waitKey(20)
    if key == 27: # exit on ESC
        break
        
camera.release()  
cv2.destroyAllWindows()        
        

In [9]:
# using web camera 
import cv2
cv2.namedWindow("webcam test")
cam_url='http://10.0.0.67:8081/video'
print cam_url

# screenshot
import os
from PIL import Image
from PIL import ImageGrab
import cv2
x = 345
y = 281
m = 462
n = 327
 
k = 54
print '!'
while True:
    box = (x,y,m,n)
    
    img = ImageGrab.grab(box)
    cv2.imshow("webcam test", img)
    #img.show()
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): 
        break
cv2.destroyAllWindows()        
        


In [ ]:
# face detection small example
import cv2
import numpy as np 
# the path of the image
imagepath = r'./heat.jpg'
face_cascade = cv2.CascadeClassifier(r'./haarcascade_frontalface_default.xml') 
image = cv2.imread(imagepath)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
'''
faces = face_cascade.detectMultiScale(
    gray,
    scaleFactor = 1.15,
    minNeighbors = 5,
    minSize = (5,5),
    flags = cv2.cv.CV_HAAR_SCALE_IMAGE
)
'''
faces = face_cascade.detectMultiScale(gray,1.3,5);
for(x,y,w,h) in faces:
    # cv2.rectangle(image,(x,y),(x+w,y+w),(0,255,0),2)
    cv2.circle(image,(int(np.round((x+x+w)/2)),int(np.round((y+y+h)/2))),int(np.round(w/2)),(0,255,0),2)
    
cv2.imshow("Find Faces!",image)
cv2.waitKey(0)


In [33]:
# detection of eyes
import cv2
import numpy as np 
# the path of the image
imagepath = r'./detect_eye.jpg'
face_cascade = cv2.CascadeClassifier(r'./haarcascade_eye.xml') 
image = cv2.imread(imagepath)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray,1.3,5);
for(x,y,w,h) in faces:
    # cv2.rectangle(image,(x,y),(x+w,y+w),(0,255,0),2)
    cv2.circle(image,(int(np.round((x+x+w)/2)),int(np.round((y+y+h)/2))),int(np.round(w/2)),(0,255,0),2)
    
cv2.imshow("Find Faces!",image)
cv2.waitKey(0)

         